# **Entregable semana 1**
**1. Elegir una API pública**

Para este proyecto, se eligió la API de OpenWeather, una plataforma gratuita que proporciona información meteorológica en tiempo real y pronósticos futuros para cualquier ciudad del mundo.

**Datos que entrega:** Temperatura actual, sensación térmica, humedad, presión atmosférica, velocidad y dirección del viento, así como una descripción del estado del clima. Además, permite obtener un pronóstico detallado para los próximos cinco días con datos cada 3 horas.

**Acceso**: La API requiere autenticación mediante una API Key gratuita que se obtiene tras crear una cuenta en openweathermap.org. Las consultas se hacen mediante peticiones HTTP GET con parámetros como la ciudad (q), unidades (units), idioma (lang) y la clave (appid).

**Utilidad**: Esta información permite construir una herramienta que muestre el clima actual y predicciones futuras, lo que es útil para planear actividades, emitir alertas o realizar análisis climáticos por ciudad.

**2. Definir el problema**

**El problema a resolver es**:
“¿Cómo puedo consultar el estado del clima actual y el pronóstico de cualquier ciudad, desde la terminal, de forma rápida y exportable para análisis posteriores?”

Este problema es común para usuarios que necesitan monitorear condiciones meteorológicas sin depender de navegadores o interfaces gráficas, como usuarios de servidores, desarrolladores o investigadores. También se busca que los datos sean exportables a CSV y JSON para análisis posterior o almacenamiento.

**3. Crear un script funcional**

Se desarrolló un script en Python que:

-Se conecta a la API de OpenWeather utilizando requests.

-Consulta y obtiene datos tanto del clima actual como del pronóstico de 5 días.

-Valida la respuesta mediante verificación del código HTTP.

-Usa funciones reutilizables y estructuradas para mejorar la legibilidad y el mantenimiento del código.

-Además, permite cambiar la ciudad manualmente, alternar entre grados Celsius y Fahrenheit, y ofrece una detección automática de ciudad vía IP.



In [ ]:
!pip install requests
!pip install plotext
!pip install openpyxl

In [ ]:
import requests
from datetime import datetime

API_KEY = "f0f0c07c117721896152aeca0fbbe31a"

# 🌐 Obtener ubicación por IP
def obtener_ciudad_por_ip():
    try:
        res = requests.get("http://ip-api.com/json/")
        if res.status_code == 200:
            data = res.json()
            return data.get("city", "")
    except Exception as e:
        print(f"Error al obtener ubicación por IP: {e}")
    return ""

# 🌤️ Obtener clima actual
def obtener_clima_actual(ciudad, unidades):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={ciudad}&appid={API_KEY}&units={unidades}&lang=es"
    res = requests.get(url)
    if res.status_code == 200:
        return res.json()
    return None

# 📅 Obtener pronóstico
def obtener_pronostico(ciudad, unidades):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={ciudad}&appid={API_KEY}&units={unidades}&lang=es"
    res = requests.get(url)
    if res.status_code == 200:
        return res.json()
    return None

# 🔁 Convertir unidades
def cambiar_unidad(unidad_actual):
    return "imperial" if unidad_actual == "metric" else "metric"

# 🖨️ Mostrar clima
def mostrar_clima(ciudad, unidad):
    datos_actual = obtener_clima_actual(ciudad, unidad)
    datos_pronostico = obtener_pronostico(ciudad, unidad)

    if not datos_actual or not datos_pronostico:
        print("❌ No se pudo obtener información del clima.")
        return

    # Símbolos de unidad
    temp_unit = "°C" if unidad == "metric" else "°F"
    speed_unit = "m/s" if unidad == "metric" else "mph"

    print(f"\n🌍 Clima en {datos_actual['name']}, {datos_actual['sys']['country']}")
    print(f"🌡️ Temperatura: {datos_actual['main']['temp']}{temp_unit}")
    print(f"🥵 Sensación térmica: {datos_actual['main']['feels_like']}{temp_unit}")
    print(f"💧 Humedad: {datos_actual['main']['humidity']}%")
    print(f"📈 Presión: {datos_actual['main']['pressure']} hPa")
    print(f"🌬️ Viento: {datos_actual['wind']['speed']} {speed_unit}, dirección {datos_actual['wind'].get('deg', 0)}°")
    print(f"📝 Estado: {datos_actual['weather'][0]['description'].capitalize()}")

    print("\n📆 Pronóstico para los próximos días:")
    listado = datos_pronostico["list"]
    dias_mostrados = set()

    for item in listado:
        fecha = datetime.strptime(item["dt_txt"], "%Y-%m-%d %H:%M:%S")
        dia = fecha.strftime("%A %d/%m")

        # Mostrar solo una vez por día (aprox. al mediodía)
        if dia not in dias_mostrados and "12:00:00" in item["dt_txt"]:
            dias_mostrados.add(dia)
            print(f"\n🗓️ {dia}")
            print(f"   🌡️ Temp: {item['main']['temp']}{temp_unit}")
            print(f"   📝 Estado: {item['weather'][0]['description'].capitalize()}")

# 🧭 Menú principal
def main():
    print("🌦️ Bienvenido a la aplicación del clima en terminal 🌦️")

    unidad = "metric"
    ciudad = obtener_ciudad_por_ip()

    if not ciudad:
        ciudad = input("❓ No se pudo detectar tu ciudad. Ingresa una ciudad: ")
    else:
        print(f"📍 Ciudad detectada automáticamente: {ciudad}")

    while True:
        mostrar_clima(ciudad, unidad)
        print("\nOpciones:")
        print("1. Cambiar ciudad")
        print("2. Cambiar unidad (°C / °F)")
        print("3. Actualizar datos")
        print("4. Salir")

        opcion = input("👉 Elige una opción: ")

        if opcion == "1":
            ciudad = input("📝 Ingresa una nueva ciudad: ")
        elif opcion == "2":
            unidad = cambiar_unidad(unidad)
        elif opcion == "3":
            continue
        elif opcion == "4":
            print("👋 ¡Hasta luego!")
            break
        else:
            print("❌ Opción no válida.")

if __name__ == "__main__":
    main()
